In [1]:
import requests
import os
import json
import pandas as pd
import numpy as np
import pickle as pkl
import TwitterUtils as TU
import seaborn as sns
import re

# import spacy
# import spacy_langdetect as sld

In [2]:
with open("user_data_temp.pkl", "rb") as file: # Unique Places
    convos = pkl.load(file)

In [3]:
pd.DataFrame(convos).head(20)

,user_id,tweet_id,tweet_text,referenced_tweets,convo_id,reply_to_user_id
0,1013303513449836547,1638630321473335297,@1shankarsharma @Pavanasoonu Think.... astrolo...,"[{'type': 'replied_to', 'id': '163774835712786...",1637733079002537986,2989553946
1,960759818,1638112643293511681,"@Pavanasoonu @1shankarsharma sir, dont bother ...","[{'type': 'replied_to', 'id': '163774965390627...",1637733079002537986,1051074201082388480
2,721002729578094592,1638034126531104768,@Pavanasoonu Pranaam Sir🙏\nWhat is your take o...,"[{'type': 'replied_to', 'id': '163773307900253...",1637733079002537986,1051074201082388480
3,751663829264666624,1638031325126074369,@1shankarsharma @Pavanasoonu If someone asks w...,"[{'type': 'replied_to', 'id': '163774835712786...",1637733079002537986,2989553946
4,1051074201082388480,1638027677776113670,@Gajendr70729189 @amitsharma2704 @1shankarshar...,"[{'type': 'replied_to', 'id': '163802387093334...",1637733079002537986,1420750442577891335
5,1420750442577891335,1638023870933344256,@Pavanasoonu @amitsharma2704 @1shankarsharma N...,"[{'type': 'replied_to', 'id': '163785066821496...",1637733079002537986,1051074201082388480
6,1399940033784684544,1638018184262606849,"@1shankarsharma @Pavanasoonu Also, the person ...","[{'type': 'replied_to', 'id': '163774835712786...",1637733079002537986,2989553946
7,1507631574451322880,1638003069635153921,"@1shankarsharma @Pavanasoonu First of all, bel...","[{'type': 'replied_to', 'id': '163774835712786...",1637733079002537986,2989553946
8,1013303513449836547,1638630321473335297,@1shankarsharma @Pavanasoonu Think.... astrolo...,"[{'type': 'replied_to', 'id': '163774835712786...",1637733079002537986,2989553946
9,960759818,1638112643293511681,"@Pavanasoonu @1shankarsharma sir, dont bother ...","[{'type': 'replied_to', 'id': '163774965390627...",1637733079002537986,1051074201082388480


## Combining Data

The `GetPlaces.py`, `GetTweets.py`, and `SampleUser.py` files have generated the following output files:
* users.json : contains all the user specific data from the 51,000 sampled users
* places.pkl : metadata related to all twitter places in the user sample
* tweets.pkl : retrieved 100 tweets from each user

In [4]:
URL_regex = 'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&\/\/=]*)'
twitter_username_re = '(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9-_]+)'
twitter_username_re = r"((^|[^@\w])@(\w{1,15})\b)*"

In [6]:
with open("places.pkl", "rb") as file: # Unique Places
    places = pkl.load(file)

with open('tweets.pkl', 'rb') as file: # Rename to Tweets, user_id, place_id
    data = pkl.load(file)

with open('users.json', 'r') as file: # Actual data for user accounts
    user_json = file.read()

test = '{"total": [' + user_json.replace("}{", "},{") + "]}"
user_data = json.loads(test)
users = [u['data'] for u in user_data["total"]]
flat_list = [user_id for user in users for user_id in user]

users_df = pd.DataFrame(flat_list).rename(columns = {'name' : 'user_name_field'})

In [7]:
places_unpacked = [item  for item in places.values()]
def unpack_place(place):
    return (place.id, place.name, place.full_name, place.country, place.country_code, place.place_type, place.centroid[0], place.centroid[1])

unpacked_places = [unpack_place(place) for place in places_unpacked]
place_df = pd.DataFrame(unpacked_places, columns = ("id", "name", "full_name", "country", "country_code", "type", "lat", "lon"))

In [ ]:
place_df

,id,name,full_name,country,country_code,type,lat,lon
0,67b98f17fdcf20be,Boston,"Boston, MA",United States,US,city,-71.088478,42.311398
1,5c62ffb0f0f3479d,Phoenix,"Phoenix, AZ",United States,US,city,-112.073523,33.552088
2,8a927a7056322151,Botswana,Botswana,Botswana,BW,country,24.481179,-22.342707
3,06168d1feda43857,South East,"South East, England",United Kingdom,GB,admin,-0.993491,51.450970
4,94d47cc557aa35f4,Nicholasville,"Nicholasville, KY",United States,US,city,-84.579594,37.872506
...,...,...,...,...,...,...,...,...
7364,486bc865d347cd73,Florin,"Florin, CA",United States,US,city,-121.405325,38.489315
7365,35f73f0698fa5f98,Athens,"Athens, TN",United States,US,city,-84.603682,35.445604
7366,00ace66c41e736ad,Mubi,"Mubi, Nigeria",Nigeria,NG,city,13.269995,10.268770
7367,fb4d9b85c6daedc1,Nigel,"Nigel, South Africa",South Africa,ZA,city,28.464121,-26.390979


URL_regex = 'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)'

In [ ]:
URL_regex = r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&\/\/=]*)'

In [ ]:
df = pd.DataFrame(data)
merged = pd.merge(df, place_df, how = 'left', left_on = 'place_id', right_on='id')
full_data = pd.merge(merged, users_df, how='left', left_on ='user_id', right_on = 'id')
# full_data.to_csv('fulldata.csv', index = False) # 3 Gigs of data, not great...
# full_data.description.iloc[0,]
# places_unpacked # Cool opportunity for geographic visualization

In [ ]:
len(full_data['user_id'].unique())

30470

In [ ]:
# Limit to top 6 countries
by_country = merged.groupby('country').count()
top6 = by_country.sort_values(by = 'user_id', ascending=False).head(6)
total = by_country.user_id.sum()
top6.user_id.divide(total).sum()

0.9066735266648764

In [ ]:
full_data.columns

Index(['user_id', 'tweet_id', 'tweet_text', 'place_id', 'id_x', 'name',
       'full_name', 'country', 'country_code', 'type', 'lat', 'lon',
       'username', 'description', 'id_y', 'user_name_field', 'location',
       'withheld'],
      dtype='object')

The top 6 countries account for 91% of the total users collected, which suggests pretty good coverage. Dropping unnecessary columns

Rename Columns

In [ ]:
target_fields = ['user_id', 'tweet_id', 'tweet_text', 'place_id', 'name',
       'full_name', 'country', 'country_code', 'type', 'username',
       'description', 'user_name_field', 'location', 'lat', 'lon']
reduced_df = full_data[target_fields]
reduced_df = reduced_df.rename(columns={'name':'place_name', 
                                        'full_name':'full_place_name',
                                        'type': 'place_type', 
                                        'description':'profile_description',
                                        'user_name_field':'profile_name',
                                        'location':'profile_location',
                                        'lat': 'lat',
                                        'lon':'lon'
                                        })

Drop Duplicates

In [ ]:
top6_countries = top6.index
top6 = reduced_df[reduced_df['country'].isin(top6_countries)]
unique_tweets = top6['tweet_id'].unique()
top6 = top6.drop_duplicates('tweet_id')

Detect languages

In [ ]:
def get_lang_detector(nlp, name):
    return sld.LanguageDetector()

# Uncomment when running for first time
nlp = spacy.load("en_core_web_sm")
spacy.Language.factory('language_detector', func = get_lang_detector)
nlp.add_pipe('language_detector', last =True)

def get_language(text):
    return nlp(text)._.language['score']

In [ ]:
get_language(top6.tweet_text.iloc[0])

0.999998530043281

In [ ]:
# data_dict = top6.to_dict(orient='records')
# langs = [get_language(d['tweet_text']) for d in data_dict]

## Clean Tweet Texts

In [ ]:
text_test = top6.tweet_text.iloc[1]
text_test

'@JatinPandyaVBNJ @1shankarsharma Indeed.  Nadi Jyotish has excellent rules.  Namaste.'

In [ ]:
re.sub(URL_regex, "", text_test)
re.sub(twitter_username_re, "", text_test)

' Indeed.  Nadi Jyotish has excellent rules.  Namaste.'

In [ ]:
def clean_text(text):
    temp = re.sub(URL_regex, "", text)

    return re.sub(twitter_username_re, "", text)

top6['clean_text'] = top6.tweet_text.apply(clean_text)

In [ ]:
top6 = top6.drop_duplicates('tweet_id')
top6.shape

In [ ]:
top6.to_csv('filtered_data.csv', index=False)